# Imports

In [1]:
import pandas as pd
import numpy as np
import spacy
import nltk
import re

## Helper Functions for wordNgram

In [2]:
# Helper functions for wordNgram
def boolArray(test_array):
    if False in test_array:
        return False
    else:
        return True
    
def resolve(gram,constraint,resolvedConstraints):
    try: 
        constraint_mode = constraint[2]
    except:
        constraint_mode = "orth"
        
    if constraint_mode.lower() == "orth":
        if constraint[0].startswith("~"):
            resolvedConstraints.append(not bool(re.search(r'{}'.format(constraint[0][1:]),gram['Orth'][constraint[1]-1])))
        else:
            resolvedConstraints.append(bool(re.search(r'{}'.format(constraint[0]),gram['Orth'][constraint[1]-1])))
    elif constraint_mode.lower() == "lemma":
        if constraint[0].startswith("~"):
            resolvedConstraints.append(not bool(re.search(r'{}'.format(constraint[0][1:]),gram['Lemmata'][constraint[1]-1])))
        else:
            resolvedConstraints.append(bool(re.search(r'{}'.format(constraint[0]),gram['Lemmata'][constraint[1]-1])))
    elif constraint_mode.lower() == "fg":
        if constraint[0].startswith("~"):
            resolvedConstraints.append(not bool(re.search(r'{}'.format(constraint[0][1:]),gram['FG_POS'][constraint[1]-1])))
        else:
            resolvedConstraints.append(bool(re.search(r'{}'.format(constraint[0]),gram['FG_POS'][constraint[1]-1])))
    elif constraint_mode.lower() == "cg":
        if constraint[0].startswith("~"):
            resolvedConstraints.append(not bool(re.search(r'{}'.format(constraint[0][1:]),gram['CG_POS'][constraint[1]-1])))
        else:
            resolvedConstraints.append(bool(re.search(r'{}'.format(constraint[0]),gram['CG_POS'][constraint[1]-1])))

def resolveConstraints(gram, constraints):
    resolvedConstraints = []
    if isinstance(constraints,(list,)):
        for constraint in constraints:
            resolve(gram,constraint,resolvedConstraints)
    elif isinstance(constraints,(tuple,)):  
        constraint = constraints
        resolve(gram,constraint,resolvedConstraints)
    else:
        print("Input error.")
        
    return resolvedConstraints
        

## Main function

In [3]:
# ngram function
# WICHTIG: word und constraint nehmen jetzt RegExe entgegen, d.h. soll nach einem einzigen Wort gesucht werden 
# muss der RegEx gegebenenfalls mit Zeilenanfang "^" und Zeilenende "$" angegeben werden um eine "Fuzzy Search" auszuschließen
def wordNgram(text, # Erwartet eine Liste von Strings.
              word=False, # Das Wort nach dessen Kollokation gesucht wird, (word, position, mode). Mode may be
                          # 'orth' (default), 'lemma', 'FG' (fine grained POS), 'CG' (searchphrase grained POS).
              n=2, # Gibt die Anzahl der Grame an; 2 = Bigram, 3 = Trigram, etc.
              threshold=0, # Unter welcher Häufigkeit Gramme nicht mehr angezeigt werden soll
              constraint=False, # Ob ein anderes Wort mit spezifischer Position im Gram als 
                                # Constraint der Ergebnismenge dienen soll, e.g. (searchphrase, position, mode)
                                # constraint=("^muss$",2) => "muss" soll als zweites Wort im Gram auftauchen
                                # soll ein Constraint nicht mit in die Ergebnismenge aufgenommen werden, muss
                                # das Suchwort mit "~" präfigiert werden. Als Mode kann wie bei word 
                                # 'orth' (default), 'lemma', 'FG' (fine grained POS), 'CG' (coarse grained POS) angegeben
                                # werden.
              source="spacy",   # Source mode of text. By default a spacy token document is expected.
              boilDown=False # True converts all found ngrams to lower case. Thus the frequencies of ngrams 
                             # only distinguishable by case are summed up.
             ):
    ngrams_ = nltk.ngrams(text, n)
    
    ngram_list = []
    ngram_frequ = {}
    
    for grams in list(ngrams_):
        gram = {
            "Orth" : False,
            "FG_POS" : False,
            "CG_POS" : False,
            "Lemmata" : False
        }
        
        if source.strip().lower() == "spacy":
            gram["Orth"] = [token.orth_.strip() for token in grams]
            gram["FG_POS"] = [token.tag_.strip() for token in grams]
            gram["CG_POS"] = [token.pos_.strip() for token in grams]
            gram["Lemmata"] = [token.lemma_.strip() for token in grams]
        else:
            gram["Orth"] = [token.strip() for token in grams]
            
        gram_str = " ".join(gram['Orth'])
        
        if word != False:
            if isinstance(word[1],(int,)) == True and bool(constraint) == True:
                if boolArray(resolveConstraints(gram, word)) and boolArray(resolveConstraints(gram, constraint)):
                    ngram_list.append(gram_str)     
            elif isinstance(word[1],(int,)) == True:
                if boolArray(resolveConstraints(gram, word)):
                    ngram_list.append(gram_str)
            else:
                if True in [bool(re.search(r'{}'.format(word),i)) for i in gram['Orth'] ]:
                    ngram_list.append(gram_str)
        else:
            ngram_list.append(gram_str)
            
    if boilDown == True:
        ngram_list = [ ngram.lower() for ngram in ngram_list ]
        
    for ngram in set(ngram_list):
        count = ngram_list.count(ngram)
        if count > threshold:
            ngram_frequ[ngram] = count

    for entry in sorted(ngram_frequ, key=ngram_frequ.get, reverse=True):
        print("{} => {}".format(entry,ngram_frequ[entry]))
        
    return ngram_frequ

# How to use

## Load the Spacy Language Class

In [4]:
German = spacy.load('de_core_news_sm')
English = spacy.load('en_core_web_sm')

## Load the text

In [17]:
# The Phoenix on the Sword. (2012, November 9). In Wikisource . Retrieved 15:17, September 26, 2021, from https://en.wikisource.org/w/index.php?title=The_Phoenix_on_the_Sword&oldid=4132475
text = """"Know, oh prince, that between the years when the oceans drank Atlantis and the gleaming cities, and the years of the rise of the Sons of Aryas, there was an Age undreamed of, when shining kingdoms lay spread across the world like blue mantles beneath the stars — Nemedia, Ophir, Brythunia, Hyperborea, Zamora with its dark-haired women and towers of spider-haunted mystery, Zingara with its chivalry, Koth that bordered on the pastoral lands of Shem, Stygia with its shadow-guarded tombs, Hyrkania whose riders wore steel and silk and gold. But the proudest kingdom of the world was Aquilonia, reigning supreme in the dreaming west. Hither came Conan, the Cimmerian, black-haired, sullen-eyed, sword in hand, a thief, a reaver, a slayer, with gigantic melancholies and gigantic mirth, to tread the jeweled thrones of the Earth under his sandalled feet."

— The Nemedian Chronicles.

Over shadowy spires and gleaming towers lay the ghostly darkness and silence that runs before dawn. Into a dim alley, one of a veritable labyrinth of mysterious winding ways, four masked figures came hurriedly from a door which a dusky hand furtively opened. They spoke not but went swiftly into the gloom, cloaks wrapped closely about them; as silently as the ghosts of murdered men they disappeared in the darkness. Behind them a sardonic countenance was framed in the partly opened door; a pair of evil eyes glittered malevolently in the gloom.

"Go into the night, creatures of the night," a voice mocked. "Oh, fools, your doom hounds your heels like a blind dog, and you know it not."

The speaker closed the door and bolted it, then turned and went up the corridor, candle in hand. He was a somber giant, whose dusky skin revealed his Stygian blood. He came into an inner chamber, where a tall, lean man in worn velvet lounged like a great lazy cat on a silken couch, sipping wine from a huge golden goblet.

"Well, Ascalante," said the Stygian, setting down the candle, "your dupes have slunk into the streets like rats from their burrows. You work with strange tools."

"Tools?" replied Ascalante. "Why, they consider me that. For months now, ever since the Rebel Four summoned me from the southern desert, I have been living in the very heart of my enemies, hiding by day in this obscure house, skulking through dark alleys and darker corridors at night. And I have accomplished what those rebellious nobles could not. Working through them, and through other agents, many of whom have never seen my face, I have honeycombed the empire with sedition and unrest. In short I, working in the shadows, have paved the downfall of the king who sits throned in the sun. By Mitra, I was a statesman before I was an outlaw."

"And these dupes who deem themselves your masters?"

"They will continue to think that I serve them, until our present task is completed. Who are they to match wits with Ascalante? Volmana, the dwarfish count of Karaban; Gromel, the giant commander of the Black Legion; Dion, the fat baron of Attalus; Rinaldo, the hare-brained minstrel. I am the force which has welded together the steel in each, and by the clay in each, I will crush them when the time comes. But that lies in the future; tonight the king dies."

"Days ago I saw the imperial squadrons ride from the city," said the Stygian.

"They rode to the frontier which the heathen Picts assail — thanks to the strong liquor which I've smuggled over the borders to madden them. Dion's great wealth made that possible. And Volmana made it possible to dispose of the rest of the imperial troops which remained in the city. Through his princely kin in Nemedia, it was easy to persuade King Numa to request the presence of Count Trocero of Poitain, seneschal of Aquilonia; and of course, to do him honor, he'll be accompanied by an imperial escort, as well as his own troops, and Prospero, King Conan's right­hand man. That leaves only the king's personal bodyguard in the city — besides the Black Legion. Through Gromel I've corrupted a spendthrift officer of that guard, and bribed him to lead his men away from the king's door at midnight.

"Then, with sixteen desperate rogues of mine, we enter the palace by a secret tunnel. After the deed is done, even if the people do not rise to welcome us, Gromel's Black Legion will be sufficient to hold the city and the crown."

"And Dion thinks that crown will be given to him?"

"Yes. The fat fool claims it by reason of a trace of royal blood. Conan makes a bad mistake in letting men live who still boast descent from the old dynasty, from which he tore the crown of Aquilonia.

"Volmana wishes to be reinstated in royal favor as he was under the old regime, so that he may lift his poverty-ridden estates to their former grandeur. Gromel hates Pallantides, commander of the Black Dragons, and desires the command of the whole army, with all the stubbornness of the Bossonian. Alone of us all, Rinaldo has no personal ambition. He sees in Conan a red-handed, rough-footed barbarian who came out of the north to plunder a civilized land. He idealizes the king whom Conan killed to get the crown, remembering only that he occasionally patronized the arts, and forgetting the evils of his reign, and he is making the people forget. Already they openly sing The Lament for the King in which Rinaldo lauds the sainted villain and denounces Conan as 'that black-hearted savage from the abyss.' Conan laughs, but the people snarl."

"Why does he hate Conan?"

"Poets always hate those in power. To them perfection is always just behind the last corner, or beyond the next. They escape the present in dreams of the past and future. Rinaldo is a flaming torch of idealism, rising, as he thinks, to overthrow a tyrant and liberate the people. As for me — well, a few months ago I had lost all ambition but to raid the caravans for the rest of my life; now old dreams stir. Conan will die; Dion will mount the throne. Then he, too, will die. One by one, all who oppose me will die — by fire, or steel, or those deadly wines you know so well how to brew. Ascalante, king of Aquilonia! How like you the sound of it?"

The Stygian shrugged his broad shoulders.

"There was a time," he said with unconcealed bitterness, "when I, too, had my ambitions, beside which yours seem tawdry and childish. To what a state I have fallen! My old-time peers and rivals would stare indeed could they see Thoth-amon of the Ring serving as the slave of an outlander, and an outlaw at that; and aiding in the petty ambitions of barons and kings!"

"You laid your trust in magic and mummery," answered Ascalante carelessly. "I trust my wits and my sword."

"Wits and swords are as straws against the wisdom of the Darkness," growled the Stygian, his dark eyes flickering with menacing lights and shadows. "Had I not lost the Ring, our positions might be reversed."

"Nevertheless," answered the outlaw impatiently, "you wear the stripes of my whip on your back, and are likely to continue to wear them."

"Be not so sure!" the fiendish hatred of the Stygian glittered for an instant redly in his eyes. "Some day, somehow, I will find the Ring again, and when I do, by the serpent-fangs of Set, you shall pay—"

The hot-tempered Aquilonian started up and struck him heavily across the mouth. Thoth reeled back, blood starting from his lips.

"You grow over-bold, dog," growled the outlaw. "Have a care; I am still your master who knows your dark secret. Go upon the housetops and shout that Ascalante is in the city plotting against the king — if you dare."

"I dare not," muttered the Stygian, wiping the blood from his lips.

"No, you do not dare," Ascalante grinned bleakly. "For if I die by your stealth or treachery, a hermit priest in the southern desert will know of it, and will break the seal of a manuscript I left in his hands. And having read, a word will be whispered in Stygia, and a wind will creep up from the south by midnight. And where will you hide your head, Thoth-amon?"

The slave shuddered and his dusky face went ashen.

"Enough!" Ascalante changed his tone peremptorily. "I have work for you. I do not trust Dion. I bade him ride to his country estate and remain there until the work tonight is done. The fat fool could never conceal his nervousness before the king today. Ride after him, and if you do not overtake him on the road, proceed to his estate and remain with him until we send for him. Don't let him out of your sight. He is mazed with fear, and might bolt — might even rush to Conan in a panic, and reveal the whole plot, hoping thus to save his own hide. Go!"

The slave bowed, hiding the hate in his eyes, and did as he was bidden. Ascalante turned again to his wine. Over the jeweled spires was rising a dawn crimson as blood. """.replace('\n', ' ').replace('  ', ' ').strip()

## Process the text with the language class

In [18]:
spacyDocument = English(text)

## Use the spaCy document for your queries

### Show nGrams

`n=2` sets the length of the ngram to two.

In [21]:
wordNgram(spacyDocument, n=2)

. " => 28
, and => 18
of the => 17
in the => 13
" " => 13
, " => 13
the king => 7
, a => 6
from the => 6
? " => 6
, I => 5
the city => 5
I have => 5
, the => 5
" The => 5
. He => 5
the Stygian => 5
the people => 4
. And => 4
! " => 4
do not => 4
and the => 3
the Black => 3
of Aquilonia => 3
to his => 3
into the => 3
the crown => 3
, to => 3
in his => 3
" I => 3
will die => 3
was a => 3
blood . => 3
of a => 3
Black Legion => 3
. I => 3
the Ring => 3
of my => 3
Stygian , => 3
, or => 3
will be => 3
with its => 3
, with => 3
as he => 3
, hiding => 2
I was => 2
. Ascalante => 2
I 've => 2
them , => 2
night , => 2
" And => 2
the outlaw => 2
the old => 2
them . => 2
as the => 2
the years => 2
to the => 2
commander of => 2
dog , => 2
his eyes => 2
his lips => 2
rest of => 2
southern desert => 2
- haired => 2
lips . => 2
of it => 2
the rest => 2
an outlaw => 2
" You => 2
black - => 2
I do => 2
his own => 2
The slave => 2
fat fool => 2
I am => 2
continue to => 2
ago I => 2
from a => 2
Nemedia ,

{'our positions': 1,
 'north to': 1,
 'denounces Conan': 1,
 'or beyond': 1,
 'snarl .': 1,
 ', hiding': 2,
 'I was': 2,
 'whom Conan': 1,
 'that Ascalante': 1,
 'came out': 1,
 'the ghostly': 1,
 'world like': 1,
 'in worn': 1,
 'and an': 1,
 'of your': 1,
 'could they': 1,
 '. Ascalante': 2,
 'carelessly .': 1,
 'to dispose': 1,
 '; a': 1,
 'Black Dragons': 1,
 'upon the': 1,
 'seen my': 1,
 'dare .': 1,
 'and the': 3,
 'There was': 1,
 'you hide': 1,
 'presence of': 1,
 'wore steel': 1,
 'tempered Aquilonian': 1,
 ', sword': 1,
 'the streets': 1,
 'sandalled feet': 1,
 'tawdry and': 1,
 'his tone': 1,
 'proceed to': 1,
 'they to': 1,
 'which has': 1,
 'always just': 1,
 'empire with': 1,
 'a great': 1,
 'besides the': 1,
 'even rush': 1,
 'be accompanied': 1,
 'the Darkness': 1,
 'it was': 1,
 'a tyrant': 1,
 'for him': 1,
 "'ve smuggled": 1,
 'door and': 1,
 'master who': 1,
 'dark secret': 1,
 'blind dog': 1,
 'wits with': 1,
 'at that': 1,
 'of evil': 1,
 'own hide': 1,
 "I 've":

## Specific co-occurrences

When you want to search for specific co-occurrences, just provide an tuple as argument on position two or as named argument `word`, that details the pivot for the co-occurrence. 

```python
(
    'Conan',
    1
)
```

The word whose collocation is being searched for (word, position, mode). `mode` can be 'orth' (standard), 'lemma', 'FG' (fine-grained POS), 'CG' (search term-grained POS).

The available POS tags for coarse grained tagging are shared among languages: https://universaldependencies.org/docs/u/pos/

The available POS tags for fine grained tagging differ among languages. You'll have to look it up in the spaCy model's documentation: https://spacy.io/models/de

In [24]:
wordNgram(spacyDocument, word=('Conan',1), n=2)

Conan killed => 1
Conan a => 1
Conan 's => 1
Conan laughs => 1
Conan as => 1
Conan in => 1
Conan ? => 1
Conan will => 1
Conan , => 1
Conan makes => 1


{'Conan killed': 1,
 'Conan a': 1,
 "Conan 's": 1,
 'Conan laughs': 1,
 'Conan as': 1,
 'Conan in': 1,
 'Conan ?': 1,
 'Conan will': 1,
 'Conan ,': 1,
 'Conan makes': 1}

In [26]:
wordNgram(spacyDocument, ('PROPN', 2, 'CG'), n=4, boilDown=True)

of aryas , there => 1
black legion . through => 1
, brythunia , hyperborea => 1
king conan 's right­hand => 1
, gromel 's black => 1
by mitra , i => 1
see thoth - amon => 1
? volmana , the => 1
of aquilonia ; and => 1
the ring serving as => 1
's black legion will => 1
, ophir , brythunia => 1
the earth under his => 1
the nemedian chronicles . => 1
came conan , the => 1
and volmana made it => 1
hate conan ? " => 1
, thoth - amon => 1
nemedian chronicles . over => 1
. rinaldo is a => 1
hates pallantides , commander => 1
, koth that bordered => 1
" ascalante grinned bleakly => 1
, stygia with its => 1
the black legion ; => 1
which rinaldo lauds the => 1
, ascalante , " => 1
. thoth reeled back => 1
the darkness , " => 1
of count trocero of => 1
worn velvet lounged like => 1
of poitain , seneschal => 1
, zingara with its => 1
, zamora with its => 1
trust dion . i => 1
the stygian glittered for => 1
whom conan killed to => 1
the lament for the => 1
of set , you => 1
an age undreamed of => 1

{'of aryas , there': 1,
 'black legion . through': 1,
 ', brythunia , hyperborea': 1,
 "king conan 's right\xadhand": 1,
 ", gromel 's black": 1,
 'by mitra , i': 1,
 'see thoth - amon': 1,
 '? volmana , the': 1,
 'of aquilonia ; and': 1,
 'the ring serving as': 1,
 "'s black legion will": 1,
 ', ophir , brythunia': 1,
 'the earth under his': 1,
 'the nemedian chronicles .': 1,
 'came conan , the': 1,
 'and volmana made it': 1,
 'hate conan ? "': 1,
 ', thoth - amon': 1,
 'nemedian chronicles . over': 1,
 '. rinaldo is a': 1,
 'hates pallantides , commander': 1,
 ', koth that bordered': 1,
 '" ascalante grinned bleakly': 1,
 ', stygia with its': 1,
 'the black legion ;': 1,
 'which rinaldo lauds the': 1,
 ', ascalante , "': 1,
 '. thoth reeled back': 1,
 'the darkness , "': 1,
 'of count trocero of': 1,
 'worn velvet lounged like': 1,
 'of poitain , seneschal': 1,
 ', zingara with its': 1,
 ', zamora with its': 1,
 'trust dion . i': 1,
 'the stygian glittered for': 1,
 'whom conan kill

You may use regex patterns on  the string position:

In [31]:
wordNgram(spacyDocument, ('ma[kd]e[s]{0,1}',2), n=3, boilDown=True)

wealth made that => 1
volmana made it => 1
conan makes a => 1


{'wealth made that': 1, 'volmana made it': 1, 'conan makes a': 1}

## Adding further constraints

Whether another word with a specific position in the ngram should serve as a constraint of the result set, e.g. (searchphrase, position, mode) constraint=("^must$",2) => "must" should appear as the second word in the gram If a constraint is not to be included in the result set, the search word must be prefigured with "~". As with word, 'orth' (default), 'lemma', 'FG' (fine grained POS), 'CG' (coarse grained POS) can be specified as the mode.

In [34]:
# Forces a Proper Noun on position one and an Auxiliary Verb on position 2: 
wordNgram(spacyDocument, ('PROPN', 1, 'CG'), n=3, constraint=('AUX', 2, 'CG'), boilDown=True)

ascalante is in => 1
rinaldo has no => 1
rinaldo is a => 1


{'ascalante is in': 1, 'rinaldo has no': 1, 'rinaldo is a': 1}

In [35]:
# Forces a Proper Noun on position one and NO Punctuation (`~PUNCT`) on position 3: 
wordNgram(spacyDocument, ('PROPN', 1, 'CG'), n=3, constraint=('~PUNCT', 3, 'CG'), boilDown=True)

thoth - amon => 2
gromel i 've => 1
trocero of poitain => 1
stygia with its => 1
rinaldo is a => 1
poitain , seneschal => 1
zamora with its => 1
conan a red => 1
conan 's right­hand => 1
king conan 's => 1
conan will die => 1
pallantides , commander => 1
ascalante is in => 1
dion . i => 1
conan in a => 1
ring , our => 1
gromel hates pallantides => 1
earth under his => 1
bossonian . alone => 1
aquilonia ! how => 1
count trocero of => 1
nemedia , ophir => 1
hither came conan => 1
supreme in the => 1
dion will mount => 1
ascalante grinned bleakly => 1
aquilonian started up => 1
stygia , and => 1
ascalante turned again => 1
attalus ; rinaldo => 1
hyrkania whose riders => 1
sons of aryas => 1
rebel four summoned => 1
king numa to => 1
ascalante ? volmana => 1
karaban ; gromel => 1
ascalante changed his => 1
rinaldo has no => 1
rinaldo lauds the => 1
zingara with its => 1
stygian shrugged his => 1
brythunia , hyperborea => 1
ascalante , king => 1
aryas , there => 1
stygian , his => 1
hyperbo

{"gromel i 've": 1,
 'trocero of poitain': 1,
 'stygia with its': 1,
 'rinaldo is a': 1,
 'poitain , seneschal': 1,
 'zamora with its': 1,
 'conan a red': 1,
 "conan 's right\xadhand": 1,
 "king conan 's": 1,
 'conan will die': 1,
 'pallantides , commander': 1,
 'ascalante is in': 1,
 'dion . i': 1,
 'conan in a': 1,
 'ring , our': 1,
 'gromel hates pallantides': 1,
 'earth under his': 1,
 'bossonian . alone': 1,
 'aquilonia ! how': 1,
 'count trocero of': 1,
 'nemedia , ophir': 1,
 'hither came conan': 1,
 'supreme in the': 1,
 'dion will mount': 1,
 'ascalante grinned bleakly': 1,
 'aquilonian started up': 1,
 'stygia , and': 1,
 'ascalante turned again': 1,
 'attalus ; rinaldo': 1,
 'thoth - amon': 2,
 'hyrkania whose riders': 1,
 'sons of aryas': 1,
 'rebel four summoned': 1,
 'king numa to': 1,
 'ascalante ? volmana': 1,
 'karaban ; gromel': 1,
 'ascalante changed his': 1,
 'rinaldo has no': 1,
 'rinaldo lauds the': 1,
 'zingara with its': 1,
 'stygian shrugged his': 1,
 'brythuni

In [36]:
wordNgram(spacyDocument, ('PROPN', 1, 'CG'), n=3, constraint=[('~PUNCT', 2, 'CG'), ('~PUNCT', 3, 'CG')], boilDown=True)

aquilonian started up => 1
gromel i 've => 1
ascalante turned again => 1
trocero of poitain => 1
age undreamed of => 1
hyrkania whose riders => 1
sons of aryas => 1
gromel 's black => 1
dion thinks that => 1
stygia with its => 1
rebel four summoned => 1
numa to request => 1
rinaldo is a => 1
conan makes a => 1
zamora with its => 1
king numa to => 1
conan a red => 1
koth that bordered => 1
atlantis and the => 1
ascalante changed his => 1
rinaldo has no => 1
king in which => 1
rinaldo lauds the => 1
thoth reeled back => 1
conan 's right­hand => 1
king conan 's => 1
conan will die => 1
ascalante is in => 1
conan in a => 1
conan killed to => 1
legion will be => 1
black legion will => 1
earth under his => 1
gromel hates pallantides => 1
lament for the => 1
zingara with its => 1
count trocero of => 1
velvet lounged like => 1
hither came conan => 1
supreme in the => 1
stygian shrugged his => 1
amon of the => 1
ring serving as => 1
stygian glittered for => 1
dion will mount => 1
ascalante grin

{'aquilonian started up': 1,
 "gromel i 've": 1,
 'ascalante turned again': 1,
 'trocero of poitain': 1,
 'age undreamed of': 1,
 'hyrkania whose riders': 1,
 'sons of aryas': 1,
 "gromel 's black": 1,
 'dion thinks that': 1,
 'stygia with its': 1,
 'rebel four summoned': 1,
 'numa to request': 1,
 'rinaldo is a': 1,
 'conan makes a': 1,
 'zamora with its': 1,
 'king numa to': 1,
 'conan a red': 1,
 'koth that bordered': 1,
 'atlantis and the': 1,
 'ascalante changed his': 1,
 'rinaldo has no': 1,
 'king in which': 1,
 'rinaldo lauds the': 1,
 'thoth reeled back': 1,
 "conan 's right\xadhand": 1,
 "king conan 's": 1,
 'conan will die': 1,
 'ascalante is in': 1,
 'conan in a': 1,
 'conan killed to': 1,
 'legion will be': 1,
 'black legion will': 1,
 'earth under his': 1,
 'gromel hates pallantides': 1,
 'lament for the': 1,
 'zingara with its': 1,
 'count trocero of': 1,
 'velvet lounged like': 1,
 'hither came conan': 1,
 'supreme in the': 1,
 'stygian shrugged his': 1,
 'amon of the':

In [37]:
wordNgram(spacyDocument, ('ADJ', 3, 'CG'), n=3, constraint=('AUX', 2, 'CG'), boilDown=True)

and are likely => 1
it was easy => 1
he was bidden => 1
will be sufficient => 1


{'and are likely': 1,
 'it was easy': 1,
 'he was bidden': 1,
 'will be sufficient': 1}

Using regex patternms allows to chain POS tags together in a query.

In [39]:
wordNgram(spacyDocument, ('(PROPN|NOUN|PRON)', 3, 'CG'), n=3, constraint=('AUX', 2, 'CG'), boilDown=True)

who are they => 1
to do him => 1
why does he => 1
i have work => 1
world was aquilonia => 1
" had i => 1


{'who are they': 1,
 'to do him': 1,
 'why does he': 1,
 'i have work': 1,
 'world was aquilonia': 1,
 '" had i': 1}

In [42]:
wordNgram(spacyDocument, ('(PROPN|NOUN|PRON)', 1, 'CG'), n=2, constraint=('VERB', 2, 'CG'), boilDown=True)

you know => 2
i will => 2
we send => 1
stygian shrugged => 1
time comes => 1
you dare => 1
dreams stir => 1
eyes flickering => 1
gromel hates => 1
who deem => 1
who came => 1
you work => 1
they will => 1
ring serving => 1
they disappeared => 1
who knows => 1
doom hounds => 1
you grow => 1
speaker closed => 1
velvet lounged => 1
i die => 1
rivals would => 1
you hide => 1
i bade => 1
people forget => 1
stygian glittered => 1
king dies => 1
nobles could => 1
aquilonian started => 1
slave shuddered => 1
word will => 1
towers lay => 1
him ride => 1
volmana made => 1
legion will => 1
they escape => 1
he thinks => 1
cloaks wrapped => 1
wind will => 1
squadrons ride => 1
ascalante changed => 1
they spoke => 1
you wear => 1
he 'll => 1
they rode => 1
me will => 1
we enter => 1
he may => 1
riders wore => 1
city plotting => 1
dion will => 1
yours seem => 1
he idealizes => 1
i left => 1
conan makes => 1
they consider => 1
picts assail => 1
kingdoms lay => 1
four summoned => 1
face went => 1
figure

{'we send': 1,
 'stygian shrugged': 1,
 'time comes': 1,
 'you dare': 1,
 'dreams stir': 1,
 'eyes flickering': 1,
 'gromel hates': 1,
 'you know': 2,
 'who deem': 1,
 'who came': 1,
 'you work': 1,
 'they will': 1,
 'ring serving': 1,
 'they disappeared': 1,
 'who knows': 1,
 'doom hounds': 1,
 'you grow': 1,
 'speaker closed': 1,
 'velvet lounged': 1,
 'i die': 1,
 'rivals would': 1,
 'you hide': 1,
 'i bade': 1,
 'people forget': 1,
 'stygian glittered': 1,
 'king dies': 1,
 'nobles could': 1,
 'aquilonian started': 1,
 'slave shuddered': 1,
 'word will': 1,
 'towers lay': 1,
 'him ride': 1,
 'volmana made': 1,
 'legion will': 1,
 'they escape': 1,
 'he thinks': 1,
 'cloaks wrapped': 1,
 'wind will': 1,
 'squadrons ride': 1,
 'ascalante changed': 1,
 'they spoke': 1,
 'you wear': 1,
 "he 'll": 1,
 'they rode': 1,
 'me will': 1,
 'we enter': 1,
 'he may': 1,
 'riders wore': 1,
 'city plotting': 1,
 'dion will': 1,
 'yours seem': 1,
 'he idealizes': 1,
 'i left': 1,
 'conan makes': 1,